# Flip-n-Slide Data Augmentation

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/flipnslide_augmentation.ipynb)

This notebook demonstrates the **Flip-n-Slide** data augmentation strategy for geospatial imagery, as described in:

> Abrahams, E., Snow, T., Siegfried, M. R., & Pérez, F. (2024). *A Concise Tiling Strategy for Preserving Spatial Context in Earth Observation Imagery*. ML4RS Workshop @ ICLR 2024. [arXiv:2404.10927](https://arxiv.org/abs/2404.10927)

Flip-n-Slide is a tiling and augmentation strategy for large remote sensing images that generates overlapping tiles with diverse geometric augmentations while eliminating redundant pixel representations. The algorithm produces:

1. **Standard overlapping tiles** with half-stride, applying rotational augmentations (identity, 90°, 180°, 270°) based on grid position.
2. **Inner offset tiles** from the image interior, applying flip and rotation combinations (horizontal flip, vertical flip, 90°+hflip, 90°+vflip).

Related GitHub issue: [#88](https://github.com/opengeos/geoai/issues/88)

## Install packages

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geoai

## Using flipnslide_augmentation with a numpy array

The `flipnslide_augmentation` function accepts a numpy array of shape `(channels, height, width)` and returns augmented tiles. Let's start with a synthetic example to understand the algorithm.

In [ ]:
# Create a synthetic 3-band image (e.g., 512x512 RGB)
np.random.seed(42)
image = np.random.rand(3, 512, 512).astype(np.float32)
print(f"Input image shape: {image.shape}")

In [ ]:
# Apply Flip-n-Slide augmentation
tiles, aug_indices = geoai.flipnslide_augmentation(image, tile_size=256)
print(f"Number of tiles: {tiles.shape[0]}")
print(f"Tile shape: {tiles.shape[1:]}")
print(f"Augmentation indices: {aug_indices}")

Each augmentation index corresponds to a specific transformation:

| Index | Augmentation | Stage |
|-------|-------------|-------|
| 0 | Identity (no change) | Standard tiles |
| 1 | 180° rotation | Standard tiles |
| 2 | 90° rotation | Standard tiles |
| 3 | 270° rotation | Standard tiles |
| 4 | Horizontal flip | Inner tiles |
| 5 | Vertical flip | Inner tiles |
| 6 | 90° rotation + horizontal flip | Inner tiles |
| 7 | 90° rotation + vertical flip | Inner tiles |

### Larger image example

With a larger image we can see all 8 augmentation types in action.

In [ ]:
# Larger image to get all augmentation types
large_image = np.random.rand(3, 1024, 1024).astype(np.float32)
tiles_large, aug_large = geoai.flipnslide_augmentation(large_image, tile_size=256)

from collections import Counter
aug_counts = Counter(aug_large)
print(f"Total tiles: {len(aug_large)}")
print(f"Augmentation distribution: {dict(sorted(aug_counts.items()))}")

### Visualize sample tiles

Let's visualize one tile per augmentation type to see the transformations.

In [ ]:
aug_names = {
    0: "Identity",
    1: "180° rotation",
    2: "90° rotation",
    3: "270° rotation",
    4: "Horizontal flip",
    5: "Vertical flip",
    6: "90° rot + H-flip",
    7: "90° rot + V-flip",
}

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for aug_idx in range(8):
    ax = axes[aug_idx // 4, aug_idx % 4]
    # Find first tile with this augmentation
    tile_idx = aug_large.index(aug_idx)
    # Display RGB (transpose from CHW to HWC)
    ax.imshow(tiles_large[tile_idx].transpose(1, 2, 0))
    ax.set_title(f"{aug_names[aug_idx]}\n(index={aug_idx})", fontsize=10)
    ax.axis("off")

plt.suptitle("Flip-n-Slide Augmentation Types", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

### PyTorch tensor output

The function also supports direct PyTorch tensor output.

In [ ]:
tiles_torch, aug_torch = geoai.flipnslide_augmentation(
    image, tile_size=256, output_format="torch"
)
print(f"Output type: {type(tiles_torch)}")
print(f"Shape: {tiles_torch.shape}")
print(f"Dtype: {tiles_torch.dtype}")

## Using export_flipnslide_tiles with a GeoTIFF

For real-world geospatial workflows, `export_flipnslide_tiles` reads a raster file and exports each tile as an individual GeoTIFF with proper CRS and geotransform.

### Download sample data

In [ ]:
url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/aerial.tif"
image_path = geoai.download_file(url, "aerial.tif")
print(f"Downloaded to: {image_path}")

In [ ]:
import rasterio

with rasterio.open(image_path) as src:
    print(f"CRS: {src.crs}")
    print(f"Dimensions: {src.width} x {src.height}")
    print(f"Bands: {src.count}")
    print(f"Resolution: {src.res}")

### Export tiles

In [ ]:
stats = geoai.export_flipnslide_tiles(
    in_raster=image_path,
    out_folder="flipnslide_tiles",
    tile_size=256,
)
print(f"\nStatistics: {stats}")

### Inspect exported tiles

Each tile is a georeferenced GeoTIFF with the correct spatial reference.

In [ ]:
import os
import glob

tile_files = sorted(glob.glob("flipnslide_tiles/images/*.tif"))
print(f"Number of tile files: {len(tile_files)}")
print(f"\nFirst 5 tiles:")
for f in tile_files[:5]:
    print(f"  {os.path.basename(f)}")

In [ ]:
# Verify georeferencing of a tile
if tile_files:
    with rasterio.open(tile_files[0]) as tile_src:
        print(f"Tile CRS: {tile_src.crs}")
        print(f"Tile shape: {tile_src.width}x{tile_src.height}")
        print(f"Tile bounds: {tile_src.bounds}")
        print(f"Tile transform: {tile_src.transform}")

### Visualize exported tiles

In [ ]:
# Show a sample of exported tiles
n_show = min(8, len(tile_files))
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
    if i < n_show:
        with rasterio.open(tile_files[i]) as tile_src:
            data = tile_src.read()
        # Display as RGB (first 3 bands)
        rgb = data[:3].transpose(1, 2, 0)
        # Normalize for display
        rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8)
        ax.imshow(rgb)
        ax.set_title(os.path.basename(tile_files[i]), fontsize=8)
    ax.axis("off")

plt.suptitle("Sample Exported Flip-n-Slide Tiles", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## Using tiling_strategy parameter in export_geotiff_tiles

The existing `export_geotiff_tiles` function now supports a `tiling_strategy` parameter. Setting it to `"flipnslide"` uses the Flip-n-Slide algorithm instead of the regular grid.

In [ ]:
geoai.export_geotiff_tiles(
    in_raster=image_path,
    out_folder="flipnslide_via_export",
    tile_size=256,
    tiling_strategy="flipnslide",
)

In [ ]:
fns_tiles = glob.glob("flipnslide_via_export/images/*.tif")
print(f"Tiles generated via export_geotiff_tiles (flipnslide): {len(fns_tiles)}")

## Comparing grid vs. Flip-n-Slide tiling

Let's compare the number of tiles produced by the regular grid approach versus Flip-n-Slide.

In [ ]:
# Grid tiling (default)
geoai.export_geotiff_tiles(
    in_raster=image_path,
    out_folder="grid_tiles",
    tile_size=256,
    stride=256,
    tiling_strategy="grid",
    quiet=True,
)

grid_tiles = glob.glob("grid_tiles/images/*.tif")
fns_tiles = glob.glob("flipnslide_tiles/images/*.tif")

print(f"Grid tiling (no overlap):       {len(grid_tiles)} tiles")
print(f"Flip-n-Slide augmented tiling:  {len(fns_tiles)} tiles")
if len(grid_tiles) > 0:
    print(f"Ratio: {len(fns_tiles) / len(grid_tiles):.1f}x more tiles with Flip-n-Slide")

Flip-n-Slide produces more tiles because it uses overlapping windows with half-stride plus inner offset tiles. Each tile receives a unique augmentation (rotation or flip), so there is no redundancy between overlapping regions. This diversity improves training data quality for machine learning models.

## Summary

The Flip-n-Slide augmentation strategy provides:

- **Spatial context preservation** through overlapping tiles
- **Data diversity** through 8 distinct augmentation types
- **No redundancy** — overlapping pixels always receive different augmentations
- **Easy integration** — works with `flipnslide_augmentation()` for numpy arrays, `export_flipnslide_tiles()` for GeoTIFF export, or `tiling_strategy="flipnslide"` on the existing `export_geotiff_tiles()` function

For more details, see the [paper](https://arxiv.org/abs/2404.10927) and the [flipnslide](https://github.com/elliesch/flipnslide) package.